In [1]:
demo_input = """x=20..30, y=-10..-5"""
demo = ((20,30),(-10,-5))
demo_trajectories = [(7,2),(6,3),(9,0),(17,-4),(6,9)]

In [2]:
puzzle_input = """x=207..263, y=-115..-63"""
puzzle = ((207,263),(-115,-63))

targets = [demo,puzzle]
target = targets[0]

In [3]:
def center(target):
    x_min,x_max,y_min,y_max = components(target)
    center = ((x_min + x_max) // 2, (y_min + y_max) // 2)

    return center
    
def components(target):
    x_range, y_range = target
    x_min, x_max = x_range
    y_min, y_max = y_range
    
    return x_min,x_max,y_min,y_max

print( components(target), center(target) )

(20, 30, -10, -5) (25, -8)


In [4]:
def in_target(target,position):
    x_min,x_max,y_min,y_max = components(target)
    x,y = position
    return x >= x_min and x <= x_max and y >= y_min and y <= y_max

print(f"in_target(target,center) = {in_target(target,center(target))}, should be True")

in_target(target,center) = True, should be True


In [5]:
def missed_target(target,position):
    x_min,x_max,y_min,y_max = components(target)
    x,y = position
    return x > x_max or y < y_min

position = center(target)
print(f"missed_target(target,{position}) = {missed_target(target,position)}, should be False")

start = center(target)
deltas = [ (start[0] // 3,start[1] // 7), (start[0] // 7,start[1] // 3) ]
for delta in deltas:
    print(target,delta)
    position = (0,0)
    missed = False
    while not missed:
        position = (position[0] + delta[0], position[1] + delta[1])
        missed = missed_target(target,position)
        print(f"missed_target(target,{position}) = {missed}")


missed_target(target,(25, -8)) = False, should be False
((20, 30), (-10, -5)) (8, -2)
missed_target(target,(8, -2)) = False
missed_target(target,(16, -4)) = False
missed_target(target,(24, -6)) = False
missed_target(target,(32, -8)) = True
((20, 30), (-10, -5)) (3, -3)
missed_target(target,(3, -3)) = False
missed_target(target,(6, -6)) = False
missed_target(target,(9, -9)) = False
missed_target(target,(12, -12)) = True


In [6]:
notes = """
The probe's x position increases by its x velocity.
The probe's y position increases by its y velocity.
Due to drag, the probe's x velocity changes by 1 toward the value 0; that is, it decreases by 1 if it is greater than 0, increases by 1 if it is less than 0, or does not change if it is already 0.
Due to gravity, the probe's y velocity decreases by 1
"""

In [7]:
def test_trajectory(target,initial_trajectory):
    
    trajectory = initial_trajectory
    hit = False
    missed = False
    position = (0,0)
    positions = []
    while not hit and not missed:
        dx,dy = trajectory
        position = (position[0] + dx,position[1] + dy)
        positions.append(position)
        signx = 0 if dx == 0 else dx // dx
        dx -= signx
        dy -= 1
        trajectory = (dx,dy)
        
        if in_target(target,position):
            hit = True
        elif missed_target(target,position):
            missed = True
    
    return hit,positions

for initial_trajectory in demo_trajectories:
    hit,positions = test_trajectory(target,initial_trajectory)
    max_y = max([y for x,y in positions])
    print("hit" if hit else "missed",initial_trajectory,"-->",max_y,positions)
    

hit (7, 2) --> 3 [(7, 2), (13, 3), (18, 3), (22, 2), (25, 0), (27, -3), (28, -7)]
hit (6, 3) --> 6 [(6, 3), (11, 5), (15, 6), (18, 6), (20, 5), (21, 3), (21, 0), (21, -4), (21, -9)]
hit (9, 0) --> 0 [(9, 0), (17, -1), (24, -3), (30, -6)]
missed (17, -4) --> -4 [(17, -4), (33, -9)]
hit (6, 9) --> 45 [(6, 9), (11, 17), (15, 24), (18, 30), (20, 35), (21, 39), (21, 42), (21, 44), (21, 45), (21, 45), (21, 44), (21, 42), (21, 39), (21, 35), (21, 30), (21, 24), (21, 17), (21, 9), (21, 0), (21, -10)]


In [8]:
for target in targets:
    
    x_min,x_max,y_min,y_max = components(target)
    maximum_y = y_max
    maximum_trajectory = (0,0)
    target_center = center(target)
    cx,cy = target_center
    cx *= 3
    cy *= 3
    hit_count, missed_count = (0,0)
    for dx in range(1,max(1,cx)):
        for dy in range(min(cy,-cy),max(cy,-cy)):
            initial_trajectory = (dx,dy)
            hit,positions = test_trajectory(target,initial_trajectory)
            
            if hit:
                hit_count += 1
                max_y = max([y for x,y in positions])
                #print("hit" if hit else "missed",initial_trajectory,"-->",max_y,positions)
                if max_y > maximum_y:
                    maximum_y = max_y
                    maximum_trajectory = initial_trajectory
            else:
                #print("missed",initial_trajectory)
                missed_count += 1
                
    print(f"Target {target} Trajectory {maximum_trajectory} (of {hit_count} which hit) reaches MaxY = {maximum_y}")

Target ((20, 30), (-10, -5)) Trajectory (6, 9) (of 112 which hit) reaches MaxY = 45
Target ((207, 263), (-115, -63)) Trajectory (20, 114) (of 4973 which hit) reaches MaxY = 6555
